## Comparing different groups of experiments - drivers



This notebook compares different groups of experiments, using the metrics described in each cell.

In the cell below:
1. Change each group's folder name. It should match the folder in the `data` directory.  
2. Add experiments to each `groupX_config` dictionary.
3. Give each group a name. 

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import ttest_ind 

from pasna_analysis import DataLoader, Experiment, ExperimentConfig, Group, utils

# add and remove groups/experiments
group1_folder = '25C'
group1_config = {
    '20240611_25C': ExperimentConfig(first_peak_threshold=30, to_exclude=[4,9,12,13,15,17,18,19,21,23]), 
    '20240919_25C': ExperimentConfig(first_peak_threshold=30, to_exclude=[2,5,6,7,11,14,16,18])
}

group2_folder = 'vgat>gcamp'
group2_config  = {
    '20241023_VGAT_GCaMP': ExperimentConfig(first_peak_threshold=0, to_exclude=[4,5,6,7,13,20])
}

# load experiments
group1_experiments = {}
for exp, config in group1_config.items():
    exp_path = Path.cwd().parent.joinpath('data', group1_folder, exp)
    group1_experiments[exp] = Experiment(DataLoader(exp_path), config.first_peak_threshold, config.to_exclude, dff_strategy='local_minima')

group2_experiments = {}
for exp, config in group2_config.items():
    exp_path = Path.cwd().parent.joinpath('data', group2_folder, exp)
    group2_experiments[exp] = Experiment(DataLoader(exp_path), config.first_peak_threshold, config.to_exclude, dff_strategy='local_minima')


# give each group a name
group1_name = 'elav'
group2_name = 'vgat'

# create groups
group1 = Group(group1_name, group1_experiments)
group2 = Group(group2_name, group2_experiments)

groups = [group1, group2]

In [ ]:
'''Sample size'''

for group in groups:
    num = 0
    for exp in group.experiments.values():
        for emb in exp.embryos:
            num = num + 1
    print(group.name, num)

In [ ]:
'''Developmental times at first peak'''

# get data for each group
data = {'dev_fp': [], 'group': []}
for group in groups:
    for exp in group.experiments.values():
        num = 0
        for emb in exp.embryos:
            num = num + 1
            trace = exp.traces[emb.name]
            time_first_peak = trace.peak_times[0]
            dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
            data['dev_fp'].append(dev_time_first_peak)
            data['group'].append(group.name)

df = pd.DataFrame(data)

# set plot style
# fig, ax = plt.subplots(figsize=(10, 6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.swarmplot(data=data, x='group', y='dev_fp', hue='group', size=7)
# ax.set_title('Dev time at first burst')
ax.set_ylabel('Dev time', labelpad=padding)
ax.set_xlabel('');

# adjust axes
ax.set(ylim=(1.0, 2.4))

# t-tests
group1_data = df[df['group']==group1_name]
group2_data = df[df['group']==group2_name]

print(f'{group1_name} vs {group2_name}', ttest_ind(group1_data['dev_fp'], group2_data['dev_fp']))

# save results
# plt.savefig('onset_time.svg')

In [ ]:
'''CDF of peak developmental times'''

# get data for each group
data = {'dev_time': [], 'group': []}
for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            dev_times = [emb.get_DT_from_time(t) for t in trace.peak_times]
            data['dev_time'].extend(dev_times)
            data['group'].extend([group.name]*len(dev_times))

# set plot style
# fig, ax = plt.subplots(figsize=(10, 6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.ecdfplot(data=data, x='dev_time', hue='group')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title('CDF developmental times of peaks')
ax.set_ylabel('Proportion', labelpad=padding)
ax.set_xlabel('Dev Time', labelpad=padding)

# adjust axes
# ax.set_xlim([1.9, 2.9])

# save results
# plt.savefig('cdf.svg')

In [ ]:
'''Peak amplitudes for each episode'''
num_bursts = 15

# get data for each group
data = {'peak_amp': [], 'group': [], 'peak_idx': []}
for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            t = exp.traces[emb.name]
            for i, amp in zip(range(num_bursts), t.peak_amplitudes):
                data['peak_amp'].append(amp)
                data['group'].append(group.name)
                data['peak_idx'].append(i)

amps = pd.DataFrame(data)

# set plot style
# fig, ax = plt.subplots(figsize=(10, 6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.pointplot(data=data, x='peak_idx', y='peak_amp', hue='group', linestyle='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title(f'Burst amplitudes')
ax.set_xlabel('Burst #', labelpad=padding)
ax.set_ylabel('\u0394F/F', labelpad=padding)

# adjust axes
xticks = np.arange(0,num_bursts, 2) # label even numbers
ax.set_xticks(xticks)

# save results
# plt.savefig('amps.svg')

In [ ]:
'''Developmental time for each episode'''
num_bursts = 15

# get data for each group
data = {'group': [], 'dev_time': [], 'idx': []}
for group in groups:
    for exp in group.experiments.values():
        for emb_n, emb in enumerate(exp.embryos):
            trace = exp.traces[emb.name]
            for i, t in zip(range(num_bursts), trace.peak_times):
                data['group'].append(group.name)
                data['dev_time'].append(emb.get_DT_from_time(t))
                data['idx'].append(i)

dev_times = pd.DataFrame(data)

# set plot style
# f, ax = plt.subplots(figsize=(10,6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.pointplot(data=dev_times, x='idx', y='dev_time', hue='group', alpha=0.7, errorbar='ci', linestyle='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title('Dev time per burst')
ax.set_xlabel('Burst #', labelpad=padding)
ax.set_ylabel('Dev time', labelpad=padding)

# adjust axes
xticks = np.arange(0,num_bursts, 2) # label even numbers
ax.set_xticks(xticks)

# save results
# plt.savefig('devtimepeak.svg')

In [ ]:
'''Intervals between each episode'''
num_intervals = 15

# get data for each group
data = {'group': [], 'interval': [], 'idx': []}
for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, interval in zip(range(num_intervals), trace.peak_intervals):
                data['group'].append(group.name)
                data['interval'].append(interval/60)
                data['idx'].append(i)

amps = pd.DataFrame(data)

# set plot style
# f, ax = plt.subplots(figsize=(10,6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.pointplot(data=amps, x='idx', y='interval', hue='group', linestyle='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title('Intervals by burst')
ax.set_xlabel('Interval #', labelpad=padding)
ax.set_ylabel('Time (min)', labelpad=padding)

# adjust axes
xticks = np.arange(0, num_intervals, 2) # label even numbers
ax.set_xticks(xticks)

# save results
# plt.savefig('interval_durations.svg')

In [ ]:
'''Duration of each burst'''
num_bursts = 15

# get data
data = {'group': [], 'duration': [], 'idx': []}
for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, duration in zip(range(num_bursts), trace.peak_durations):
                data['group'].append(group.name)
                data['duration'].append(duration/60)
                data['idx'].append(i)

amps = pd.DataFrame(data)

# set plot style
# f, ax = plt.subplots(figsize=(10,6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.pointplot(data=amps, x='idx', y='duration', hue='group', linestyle ='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title('Durations by peak')
ax.set_xlabel('Burst #', labelpad=padding)
ax.set_ylabel('Time (min)', labelpad=padding)

# adjust axes
xticks = np.arange(0,num_bursts, 2) # label even numbers
ax.set_xticks(xticks)

# save results
# plt.savefig('burst_durations.svg')

In [ ]:
'''Rise times'''
num_bursts = 15

# get data
data = {'group': [], 'rise_times': [], 'idx': []}
for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, rise in zip(range(15), trace.peak_rise_times):
                data['group'].append(group.name)
                data['rise_times'].append(rise)
                data['idx'].append(i)

amps = pd.DataFrame(data)

# set plot style
# f, ax = plt.subplots(figsize=(10,6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.pointplot(data=amps, x='idx', y='rise_times', hue='group',linestyle ='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title('Peak rise times')
ax.set_xlabel('Peak #', labelpad=padding)
ax.set_ylabel('Duration (min)', labelpad=padding)

# adjust axes
xticks = np.arange(0,num_bursts, 2) # label even numbers
ax.set_xticks(xticks)

# save results
# plt.savefig('risetime.svg')

In [ ]:
'''Decay times'''
num_bursts = 15

# get data
data = {'group': [], 'decay_times': [], 'idx': []}
for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, decay in zip(range(num_bursts), trace.peak_decay_times):
                data['group'].append(group.name)
                data['decay_times'].append(decay)
                data['idx'].append(i)

amps = pd.DataFrame(data)

# set plot style
# f, ax = plt.subplots(figsize=(10,6)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
ax = sns.pointplot(data=amps, x='idx', y='decay_times', hue='group', errorbar='ci', linestyle='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title('Peak decay times')
ax.set_xlabel('Peak #', labelpad=padding)
ax.set_ylabel('Duration (min)', labelpad=padding)

# adjust axes
xticks = np.arange(0,num_bursts, 2) # label even numbers
ax.set_xticks(xticks)

# save results
# plt.savefig('decaytime.svg')

In [ ]:
'''Plots AUC, grouped by bins'''
data = {'group': [], 'auc': [], 'bin': []}

n_bins = 5
first_bin = 2
bin_width = 0.2

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            dev_time_at_peaks = emb.get_DT_from_time(trace.peak_times)
            bins = [first_bin + j*bin_width for j in range(n_bins)]
            bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
            data['group'].extend([str(group.name)]*len(trace.peak_aucs))
            data['auc'].extend(trace.peak_aucs)
            data['bin'].extend(bin_idxs)

auc = pd.DataFrame(data)

# set plot style
f, ax = plt.subplots(figsize=(10,4)) # dimensions
font_scale = 2
padding = 10
sns.set_theme(style='whitegrid', palette='colorblind', font_scale=font_scale)

# plot data
bins.append(first_bin + bin_width*n_bins) # add a last bin point to generate the labels
ax = sns.pointplot(data=data, x='bin', y='auc', hue='group', linestyle='None')
sns.move_legend(ax, "lower center",bbox_to_anchor=(.5, 1.1), ncol=3, title=None, frameon=False)
# ax.set_title(f'Binned AUC - Exp {exp.name}')
ax.set_ylabel('AUC [activity*t]', labelpad=padding)
ax.set_xlabel('Dev time', labelpad=padding)

# adjust axes
x_labels = [f"{s}~{e}" for (s, e) in zip(bins[:-1], bins[1:])]
ax.set_xticks(ticks=list(range(n_bins)), labels=x_labels)

# save results
# plt.savefig('AUC.svg')